In [1]:
import mysql.connector as MySQL
from mysql.connector import errorcode
import os
import pandas as pd
import random
import time

In [2]:
host_name = "localhost"
username = 'user1'
password = 'password'
db = "webtrust"
auth_plugin = 'mysql_native_password'
conn = None

In [3]:
def connect_to_db(host_name, username, password, auth_plugin):
    try:
        conn = MySQL.connect(host=host_name, user = username, password = password, database = db, auth_plugin=auth_plugin)
    except MySQL.errorcode as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print('Connected!')
        return conn

In [4]:
def create_table(cur):
    # PRIMARY KEY can be the URL after removing duplicates
    #autoincrement
    create_table_command = ("""CREATE TABLE IF NOT EXISTS Main (
                            URL varchar(255),
                            Credibility_rating float,
                            ad_count float,
                            ad_max_size float,
                            css_definitions float,
                            page_rank float,
                            bitly_clicks float,
                            bitly_referrers float,
                            tweets float,
                            delicious_bookmarks float,
                            fb_clicks float,
                            fb_comments float,
                            fb_likes float,
                            fb_shares float,
                            fb_total float,
                            alexa_linksin float,
                            alexa_rank float,
                            commas float,
                            dots float,
                            exclamations float,
                            questions float,
                            spelling_errors float,
                            text_complexity float,
                            smog float,
                            category varchar(255),
                            JJ float,
                            NN float,
                            DT float,
                            VB float,
                            RB float,
                            document_url_y varchar(255),
                            Controversial varchar(255)
                            )""")
    
    cur.execute(create_table_command)



In [5]:
#check if url exists
def check_if_url_exists(cur, URL):
    query = ("""SELECT URL FROM MAIN WHERE URL = %s""")
    cur.execute(query,(URL,))
    
    return cur.fetchone() is not None

In [6]:
#update row if url exists
def update_row(cur,URL,
                Credibility_rating,
                ad_count,
                ad_max_size,
                css_definitions,
                page_rank,
                bitly_clicks,
                bitly_referrers,
                tweets,
                delicious_bookmarks,
                fb_clicks,
                fb_comments,
                fb_likes,
                fb_shares,
                fb_total,
                alexa_linksin,
                alexa_rank,
                commas,
                dots,
                exclamations,
                questions,
                spelling_errors,
                text_complexity,
                smog,
                category,
                JJ,
                NN,
                DT,
                VB,
                RB,
                document_url_y,
                Controversial
                ):

    query = ("""UPDATE Main 
            SET URL = %s,
                Credibility_rating = %s,
                ad_count = s%,
                ad_max_size = s%,
                css_definitions = s%,
                page_rank = s%,
                bitly_clicks = s%,
                bitly_referrers = s%,
                tweets = s%,
                delicious_bookmarks = s%,
                fb_clicks = s%,
                fb_comments = s%,
                fb_likes = s%,
                fb_shares = s%,
                fb_total = s%,
                alexa_linksin = s%,
                alexa_rank = s%,
                commas = s%,
                dots = s%,
                exclamations = s%,
                questions = s%,
                spelling_errors = s%,
                text_complexity = s%,
                smog = s%,
                category = %s,
                JJ = s%,
                NN = s%,
                DT = s%,
                VB = s%,
                RB = s%,
                document_url_y = s%,
                Controversial = s%,
             WHERE URL = %s;""")

    var_to_update = (URL,
                Credibility_rating,
                ad_count,
                ad_max_size,
                css_definitions,
                page_rank,
                bitly_clicks,
                bitly_referrers,
                tweets,
                delicious_bookmarks,
                fb_clicks,
                fb_comments,
                fb_likes,
                fb_shares,
                fb_total,
                alexa_linksin,
                alexa_rank,
                commas,
                dots,
                exclamations,
                questions,
                spelling_errors,
                text_complexity,
                smog,
                category,
                JJ,
                NN,
                DT,
                VB,
                RB,
                document_url_y,
                Controversial)

    cur.execute(query,var_to_update)

In [7]:
#updating the database
#handling data for scalability

#check if url already exists

#######flag to determine new record or existing record

'''This particular code should probably be used for the new URLs collected since they will be live data, and will always have changingnumber of all the features'''
def update_db(cur,df):
    tmp_df = pd.DataFrame(columns = [
                    'URL',
                    'Credibility_rating',
                    'ad_count',
                    'ad_max_size',
                    'css_definitions',
                    'page_rank',
                    'bitly_clicks',
                    'bitly_referrers',
                    'tweets',
                    'delicious_bookmarks',
                    'fb_clicks',
                    'fb_comments',
                    'fb_likes',
                    'fb_shares',
                    'fb_total',
                    'alexa_linksin',
                    'alexa_rank',
                    'commas',
                    'dots',
                    'exclamations',
                    'questions',
                    'spelling_errors',
                    'text_complexity',
                    'smog',
                    'category',
                    'JJ',
                    'NN',
                    'DT',
                    'VB',
                    'RB',
                    'document_url_y',
                    'Controversial'])

    for i, row in df.iterrows():
        if check_if_url_exists(cur, row['URL']):   #If url already exists, then we will update
            update_row(cur,
                       row['URL'],
                       row['Credibility_rating'],
                       row['ad_count'],
                       row['ad_max_size'],
                       row['css_definitions'],
                       row['page_rank'],
                       row['bitly_clicks'],
                       row['bitly_referrers'],
                       row['tweets'],
                       row['delicious_bookmarks'],
                       row['fb_clicks'],
                       row['fb_comments'],
                       row['fb_likes'],
                       row['fb_shares'],
                       row['fb_total'],
                       row['alexa_linksin'],
                       row['alexa_rank'],
                       row['commas'],
                       row['dots'],
                       row['exclamations'],
                       row['questions'],
                       row['spelling_errors'],
                       row['text_complexity'],
                       row['smog'],
                       row['category'],
                       row['JJ'],
                       row['NN'],
                       row['DT'],
                       row['VB'],
                       row['RB'],
                       
                       row['document_url_y'],
                       row['Controversial']
                      )
        else:  # the URL doesn't exist so we will append to the db table
            tmp_df = tmp_df.append(row)
    
    return tmp_df

#perform update


In [8]:
#perform insert
def insert_into_table(cur,URL,
                Credibility_rating,
                ad_count,
                ad_max_size,
                css_definitions,
                page_rank,
                bitly_clicks,
                bitly_referrers,
                tweets,
                delicious_bookmarks,
                fb_clicks,
                fb_comments,
                fb_likes,
                fb_shares,
                fb_total,
                alexa_linksin,
                alexa_rank,
                commas,
                dots,
                exclamations,
                questions,
                spelling_errors,
                text_complexity,
                smog,
                category,
                JJ,
                NN,
                DT,
                VB,
                RB,
                document_url_y,
                Controversial):
    insert_query = ("""INSERT INTO Main (URL,
                Credibility_rating,
                ad_count,
                ad_max_size,
                css_definitions,
                page_rank,
                bitly_clicks,
                bitly_referrers,
                tweets,
                delicious_bookmarks,
                fb_clicks,
                fb_comments,
                fb_likes,
                fb_shares,
                fb_total,
                alexa_linksin,
                alexa_rank,
                commas,
                dots,
                exclamations,
                questions,
                spelling_errors,
                text_complexity,
                smog,
                category,
                JJ,
                NN,
                DT,
                VB,
                RB,
                document_url_y,
                Controversial)
        VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
        %s,%s,%s,%s,%s,%s,%s,%s);""")
    
    row_to_insert = (URL,
                Credibility_rating,
                ad_count,
                ad_max_size,
                css_definitions,
                page_rank,
                bitly_clicks,
                bitly_referrers,
                tweets,
                delicious_bookmarks,
                fb_clicks,
                fb_comments,
                fb_likes,
                fb_shares,
                fb_total,
                alexa_linksin,
                alexa_rank,
                commas,
                dots,
                exclamations,
                questions,
                spelling_errors,
                text_complexity,
                smog,
                category,
                JJ,
                NN,
                DT,
                VB,
                RB,
                document_url_y,
                Controversial)
    cur.execute(insert_query, row_to_insert)

In [9]:
def append_from_df_to_db(cur,df):
    for i,row in df.iterrows():
        insert_into_table(cur,
                          row['URL'],
                       row['Credibility_rating'],
                       row['ad_count'],
                       row['ad_max_size'],
                       row['css_definitions'],
                       row['page_rank'],
                       row['bitly_clicks'],
                       row['bitly_referrers'],
                       row['tweets'],
                       row['delicious_bookmarks'],
                       row['fb_clicks'],
                       row['fb_comments'],
                       row['fb_likes'],
                       row['fb_shares'],
                       row['fb_total'],
                       row['alexa_linksin'],
                       row['alexa_rank'],
                       row['commas'],
                       row['dots'],
                       row['exclamations'],
                       row['questions'],
                       row['spelling_errors'],
                       row['text_complexity'],
                       row['smog'],
                       row['category'],
                       row['JJ'],
                       row['NN'],
                       row['DT'],
                       row['VB'],
                       row['RB'],
                       
                       row['document_url_y'],
                       row['Controversial']
                          )

In [10]:
#root sql workbench password - MySQL1234
#user1 password - password

In [11]:
df = pd.read_csv('cleaned_web_trust.csv', index_col=0)

In [12]:
df

,URL,Credibility_rating,ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,...,text_complexity,smog,category,JJ,NN,DT,VB,RB,document_url_y,Controversial
0,http://unemployment.ohio.gov/,3.714286,0.0,0.0,106.0,6.0,9.0,5.0,22.0,23.0,...,6.543833,15.903189,computer_internet,121.0,938.0,184.0,338.0,52.0,gov,U
1,http://www.law.cornell.edu/wex/unemployment_co...,4.428571,4.0,75000.0,1824.0,5.0,12.0,1.0,1.0,0.0,...,3.813895,15.903189,business,35.0,168.0,59.0,71.0,5.0,edu,U
2,http://kiem-tv.com/?q=node/4395,4.000000,0.0,0.0,18.0,2.0,0.0,0.0,5.0,0.0,...,3.163160,11.208143,health,18.0,100.0,23.0,52.0,11.0,com,U
3,http://www.sciencedaily.com/releases/2012/09/1...,4.833333,9.0,75000.0,383.0,3.0,45.0,9.0,100.0,2.0,...,3.727098,14.554593,health,39.0,156.0,46.0,67.0,21.0,com,U
4,http://articles.timesofindia.indiatimes.com/20...,3.888889,2.0,75000.0,599.0,3.0,3.0,3.0,7.0,0.0,...,3.357533,14.554593,science_technology,32.0,112.0,39.0,57.0,8.0,com,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5684,http://sharesubet.com/?ref=U1786712,1.000000,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,3.983827,14.554593,computer_internet,44.0,225.0,39.0,80.0,23.0,com,U
5685,https://solidtradebank.com/index.php,2.000000,0.0,0.0,278.0,0.0,0.0,0.0,0.0,0.0,...,3.192930,13.023867,business,16.0,141.0,18.0,39.0,9.0,com,U
5688,http://cfxbu.com/index.php?ref=goldpoll,2.000000,0.0,0.0,416.0,0.0,0.0,0.0,0.0,0.0,...,3.867785,13.023867,business,48.0,270.0,37.0,65.0,11.0,com,U
5689,http://skywealth.org/forumdisplay.php?fid=4,2.000000,0.0,0.0,140.0,0.0,0.0,0.0,0.0,0.0,...,4.463884,11.208143,computer_internet,10.0,340.0,6.0,21.0,6.0,org,U


In [13]:
# df[["document_id", "ad_count","ad_max_size","","","","","","","","","","","","","","","",]] = df[["a", "b"]].apply(pd.to_numeric(downcast='integer'))
# pd.to_numeric(df, downcast='integer')

In [14]:
df.columns = df.columns.str.replace(' ','_')

In [15]:
df

,URL,Credibility_rating,ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,...,text_complexity,smog,category,JJ,NN,DT,VB,RB,document_url_y,Controversial
0,http://unemployment.ohio.gov/,3.714286,0.0,0.0,106.0,6.0,9.0,5.0,22.0,23.0,...,6.543833,15.903189,computer_internet,121.0,938.0,184.0,338.0,52.0,gov,U
1,http://www.law.cornell.edu/wex/unemployment_co...,4.428571,4.0,75000.0,1824.0,5.0,12.0,1.0,1.0,0.0,...,3.813895,15.903189,business,35.0,168.0,59.0,71.0,5.0,edu,U
2,http://kiem-tv.com/?q=node/4395,4.000000,0.0,0.0,18.0,2.0,0.0,0.0,5.0,0.0,...,3.163160,11.208143,health,18.0,100.0,23.0,52.0,11.0,com,U
3,http://www.sciencedaily.com/releases/2012/09/1...,4.833333,9.0,75000.0,383.0,3.0,45.0,9.0,100.0,2.0,...,3.727098,14.554593,health,39.0,156.0,46.0,67.0,21.0,com,U
4,http://articles.timesofindia.indiatimes.com/20...,3.888889,2.0,75000.0,599.0,3.0,3.0,3.0,7.0,0.0,...,3.357533,14.554593,science_technology,32.0,112.0,39.0,57.0,8.0,com,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5684,http://sharesubet.com/?ref=U1786712,1.000000,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,3.983827,14.554593,computer_internet,44.0,225.0,39.0,80.0,23.0,com,U
5685,https://solidtradebank.com/index.php,2.000000,0.0,0.0,278.0,0.0,0.0,0.0,0.0,0.0,...,3.192930,13.023867,business,16.0,141.0,18.0,39.0,9.0,com,U
5688,http://cfxbu.com/index.php?ref=goldpoll,2.000000,0.0,0.0,416.0,0.0,0.0,0.0,0.0,0.0,...,3.867785,13.023867,business,48.0,270.0,37.0,65.0,11.0,com,U
5689,http://skywealth.org/forumdisplay.php?fid=4,2.000000,0.0,0.0,140.0,0.0,0.0,0.0,0.0,0.0,...,4.463884,11.208143,computer_internet,10.0,340.0,6.0,21.0,6.0,org,U


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5432 entries, 0 to 5690
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   URL                  5432 non-null   object 
 1   Credibility_rating   5432 non-null   float64
 2   ad_count             5432 non-null   float64
 3   ad_max_size          5432 non-null   float64
 4   css_definitions      5432 non-null   float64
 5   page_rank            5432 non-null   float64
 6   bitly_clicks         5432 non-null   float64
 7   bitly_referrers      5432 non-null   float64
 8   tweets               5432 non-null   float64
 9   delicious_bookmarks  5432 non-null   float64
 10  fb_clicks            5432 non-null   float64
 11  fb_comments          5432 non-null   float64
 12  fb_likes             5432 non-null   float64
 13  fb_shares            5432 non-null   float64
 14  fb_total             5432 non-null   float64
 15  alexa_linksin        5432 non-null   f

In [17]:
conn = connect_to_db(host_name, username, password, auth_plugin)

Connected!


In [18]:
cur = conn.cursor()

In [19]:
create_table(cur)

In [20]:
new_df = update_db(cur,df)

In [21]:
conn.commit()

In [22]:
append_from_df_to_db(cur,new_df)

In [23]:
conn.commit()

In [24]:
new_df

,URL,Credibility_rating,ad_count,ad_max_size,css_definitions,page_rank,bitly_clicks,bitly_referrers,tweets,delicious_bookmarks,...,text_complexity,smog,category,JJ,NN,DT,VB,RB,document_url_y,Controversial
0,http://unemployment.ohio.gov/,3.714286,0.0,0.0,106.0,6.0,9.0,5.0,22.0,23.0,...,6.543833,15.903189,computer_internet,121.0,938.0,184.0,338.0,52.0,gov,U
1,http://www.law.cornell.edu/wex/unemployment_co...,4.428571,4.0,75000.0,1824.0,5.0,12.0,1.0,1.0,0.0,...,3.813895,15.903189,business,35.0,168.0,59.0,71.0,5.0,edu,U
2,http://kiem-tv.com/?q=node/4395,4.000000,0.0,0.0,18.0,2.0,0.0,0.0,5.0,0.0,...,3.163160,11.208143,health,18.0,100.0,23.0,52.0,11.0,com,U
3,http://www.sciencedaily.com/releases/2012/09/1...,4.833333,9.0,75000.0,383.0,3.0,45.0,9.0,100.0,2.0,...,3.727098,14.554593,health,39.0,156.0,46.0,67.0,21.0,com,U
4,http://articles.timesofindia.indiatimes.com/20...,3.888889,2.0,75000.0,599.0,3.0,3.0,3.0,7.0,0.0,...,3.357533,14.554593,science_technology,32.0,112.0,39.0,57.0,8.0,com,U
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5684,http://sharesubet.com/?ref=U1786712,1.000000,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,3.983827,14.554593,computer_internet,44.0,225.0,39.0,80.0,23.0,com,U
5685,https://solidtradebank.com/index.php,2.000000,0.0,0.0,278.0,0.0,0.0,0.0,0.0,0.0,...,3.192930,13.023867,business,16.0,141.0,18.0,39.0,9.0,com,U
5688,http://cfxbu.com/index.php?ref=goldpoll,2.000000,0.0,0.0,416.0,0.0,0.0,0.0,0.0,0.0,...,3.867785,13.023867,business,48.0,270.0,37.0,65.0,11.0,com,U
5689,http://skywealth.org/forumdisplay.php?fid=4,2.000000,0.0,0.0,140.0,0.0,0.0,0.0,0.0,0.0,...,4.463884,11.208143,computer_internet,10.0,340.0,6.0,21.0,6.0,org,U


In [25]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5432 entries, 0 to 5690
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   URL                  5432 non-null   object 
 1   Credibility_rating   5432 non-null   float64
 2   ad_count             5432 non-null   float64
 3   ad_max_size          5432 non-null   float64
 4   css_definitions      5432 non-null   float64
 5   page_rank            5432 non-null   float64
 6   bitly_clicks         5432 non-null   float64
 7   bitly_referrers      5432 non-null   float64
 8   tweets               5432 non-null   float64
 9   delicious_bookmarks  5432 non-null   float64
 10  fb_clicks            5432 non-null   float64
 11  fb_comments          5432 non-null   float64
 12  fb_likes             5432 non-null   float64
 13  fb_shares            5432 non-null   float64
 14  fb_total             5432 non-null   float64
 15  alexa_linksin        5432 non-null   f

In [26]:
#view data in db table

cur.execute("SELECT * FROM Main")
print(cur.fetchall())

[('http://unemployment.ohio.gov/', 3.71429, 0.0, 0.0, 106.0, 6.0, 9.0, 5.0, 22.0, 23.0, 0.0, 10.0, 12.0, 62.0, 84.0, 68.0, 6137.0, 101.0, 114.0, 1.0, 0.0, 98.0, 6.54383, 15.9032, 'computer_internet', 121.0, 938.0, 184.0, 338.0, 52.0, 'gov', 'U'), ('http://www.law.cornell.edu/wex/unemployment_compensation', 4.42857, 4.0, 75000.0, 1824.0, 5.0, 12.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9098.0, 2661.0, 15.0, 23.0, 0.0, 0.0, 2.0, 3.81389, 15.9032, 'business', 35.0, 168.0, 59.0, 71.0, 5.0, 'edu', 'U'), ('http://kiem-tv.com/?q=node/4395', 4.0, 0.0, 0.0, 18.0, 2.0, 0.0, 0.0, 5.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 185.0, 809409.0, 19.0, 18.0, 0.0, 1.0, 26.0, 3.16316, 11.2081, 'health', 18.0, 100.0, 23.0, 52.0, 11.0, 'com', 'U'), ('http://www.sciencedaily.com/releases/2012/09/120902113550.htm', 4.83333, 9.0, 75000.0, 383.0, 3.0, 45.0, 9.0, 100.0, 2.0, 0.0, 17.0, 35.0, 113.0, 165.0, 50939.0, 2549.0, 19.0, 20.0, 0.0, 0.0, 15.0, 3.7271, 14.5546, 'health', 39.0, 156.0, 46.0, 67.0, 21.0, 'com', 'U'),